In [29]:
# Import findspark and initialize.
import findspark
findspark.init()

In [30]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [31]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
spark.sparkContext.addFile(file_path)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)   #REFERENCE CHATGPT

25/01/22 03:43:24 WARN SparkContext: The path https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv has been added already. Overwriting of added paths is not supported in the current version.


In [32]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("home_sales")

In [33]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

query = """
    SELECT 
        YEAR(date) AS year,
        ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY year
"""

# Execute the query
average_price_df = spark.sql(query)

# Show the result
average_price_df.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [15]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query = """
SELECT 
    year(date) AS year_sold, 
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year(date)
ORDER BY year_sold
"""
# Execute the query
average_price_df = spark.sql(query)

# Show the result
average_price_df.show()

+---------+---------+
|year_sold|avg_price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [17]:
# 5. What is the average price of a home for each year the home was built,     #REFERENCE:  CHATGPT
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
# SQL query to calculate the average price for specified homes
query = """    #REFERENCE:  CHATGPT
SELECT 
    date_built AS year_built,
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE 
    bedrooms = 3 AND 
    bathrooms = 3 AND 
    floors = 2 AND 
    sqft_living >= 2000
GROUP BY date_built
ORDER BY year_built
"""

# Execute the query
average_price_df = spark.sql(query)

# Show the result
average_price_df.show()



+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [18]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

import time

# Start the timer
start_time = time.time()

# SQL query to calculate average price per "view" rating
query = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

# Execute the query
average_price_per_view = spark.sql(query)

# Show the result
average_price_per_view.show()

# End the timer and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.570716381072998 seconds ---


In [20]:
# 7. Cache the the temporary table home_sales.   #REFERENCE:  CHATGPT
spark.sql("CACHE TABLE home_sales")

# Verify that the table is cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' table cached? {is_cached}")


Is 'home_sales' table cached? True


In [21]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [22]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
import time

# Start the timer
start_time = time.time()

# SQL query to calculate average price per "view" rating using cached data
query = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

# Execute the query
average_price_per_view = spark.sql(query)

# Show the result
average_price_per_view.show()

# End the timer and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.3696873188018799 seconds ---


In [34]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
output_path = "home_sales_partitioned"

# Write the data partitioned by "date_built" into Parquet format
home_sales_df.write.partitionBy("date_built").parquet(output_path, mode="overwrite")    #REFERENCE:  CHATGPT


In [24]:
# 11. Read the formatted parquet data.
parquet_path = "home_sales_partitioned"

# Read the Parquet data into a DataFrame
partitioned_data = spark.read.parquet(parquet_path)

# Show the schema and the first few rows of the DataFrame
partitioned_data.printSchema()
partitioned_data.show()


root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- date_built: integer (nullable = true)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         

In [25]:
# 12. Create a temporary table for the parquet data.   #REFERENCE:  CHATGPT
partitioned_data.createOrReplaceTempView("parquet_home_sales")

# Verify the temporary table
spark.sql("SHOW TABLES").show()


+--------+------------------+-----------+
|database|         tableName|isTemporary|
+--------+------------------+-----------+
|        |        home_sales|       true|
|        |parquet_home_sales|       true|
+--------+------------------+-----------+



In [26]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.
import time

# Start the timer
start_time = time.time()

# SQL query to calculate the average price per "view" rating
query = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS avg_price
FROM parquet_home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

# Execute the query on the parquet_home_sales temporary table
average_price_per_view = spark.sql(query)

# Show the result
average_price_per_view.show()

# End the timer and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))   #REFERENCE:  CHATGPT


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.6058754920959473 seconds ---


In [27]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

# Verify that the table is uncached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' table cached? {is_cached}")   #REFERENCE:  CHATGPT


Is 'home_sales' table cached? False


In [28]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("home_sales")

# Print the result
print(f"Is 'home_sales' table cached? {is_cached}")


Is 'home_sales' table cached? False
